In [1]:
import sagemaker, boto3, json
from sagemaker import get_execution_role

aws_role = get_execution_role()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hugging Face Model configuration. https://huggingface.co/models
hub = {
    'HF_MODEL_ID': 'mdarhri00/named-entity-recognition',
    'HF_TASK': 'token-classification'
}

# Create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    transformers_version='4.26.0',
    pytorch_version='1.13.1',
    py_version='py39',
    env=hub,
    role=role,
)

# Deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1,  # Number of instances
    instance_type='ml.m5.xlarge'  # EC2 instance type
)

# Make a prediction using the deployed model
predictor.predict({
    "inputs": "My name is Sarah Jessica Parker but you can call me Jessica and I live in College Park",
})

#choppy results so could make a code that if entity is the same for two things with neighboring indexes, could that as one location or person name

#NOTE for error regarding (ResourceLimitExceeded) you have to go to SageMaker and search "increase quota" then search "ml.m5.xlarge" and request more quotas each time. Then re-run the code.

----!

[{'entity': 'Person_Name',
  'score': 0.8258283734321594,
  'index': 4,
  'word': 'Sarah',
  'start': 11,
  'end': 16},
 {'entity': 'Person_Name',
  'score': 0.8314036130905151,
  'index': 5,
  'word': 'Jessica',
  'start': 17,
  'end': 24},
 {'entity': 'Person_Name',
  'score': 0.849219560623169,
  'index': 6,
  'word': 'Parker',
  'start': 25,
  'end': 31},
 {'entity': 'Person_Name',
  'score': 0.7736506462097168,
  'index': 12,
  'word': 'Jessica',
  'start': 52,
  'end': 59},
 {'entity': 'location',
  'score': 0.7681230306625366,
  'index': 17,
  'word': 'College',
  'start': 74,
  'end': 81},
 {'entity': 'location',
  'score': 0.6825906038284302,
  'index': 18,
  'word': 'Park',
  'start': 82,
  'end': 86}]